In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import random
import os

dataRep = '../data/'
scriptRep = '../kuzushiji_recognition/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar

In [3]:
testFrac = 0.15
valFrac = 0.15

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')

raw = np.load('../data/dataset/caracterClassificationFull.npz')
image = np.expand_dims(raw['image'], 3)
charOutput = raw['characterClass']

index = np.arange(image.shape[0])
np.random.shuffle(index)
image = image[index]/255.0
charOutput = charOutput[index]

nFrac = int(image.shape[0]*(1.-testFrac))
nVal = int(image.shape[0]*(1.-testFrac-valFrac))
print(image.shape[0], image.shape[1:], nVal, nFrac)

500 (32, 32, 1) 350 425


In [4]:
def trainAndEvaluateModel(model, filenameModel, batchSize):
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=filenameModel,
                                                 monitor='val_loss',
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode='auto', period=1)
    history = keras.callbacks.History()
    reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1, patience=2,
                                                 verbose=0,
                                                 mode='auto')
    earlyStop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=1e-7, patience=5,
                                              verbose=0, mode='auto')
    callbacks = [checkpoint, history, reduceLR, earlyStop]
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(image[:nVal], charOutput[:nVal], epochs=50, batch_size=batchSize,
              validation_data=(image[nVal:nFrac], charOutput[nVal:nFrac]), shuffle=True, callbacks=callbacks)
        
    test_loss, test_acc = model.evaluate(image[nFrac:], charOutput[nFrac:])
    
    return test_loss


def optimizeHyperParameter(funcCreateModel, filenameModel, dropoutLimit=(0.,1.0), batchSizeLimit=(1,10), convolutionLayerLimit=(2,10), denseLayerLimit=(2,10), maxIter=30):
    
    i=0
    stop=False
    curentParam = [0.5, 5, 4, 8] # (dropout, batchsize, convolution, dense)
    
    stdLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])), filenameModel, 2**(curentParam[1]))
    
    while (i<maxIter) and not stop:
        i+=1
        stop=True
        
        if (curentParam[0]-0.1) >= dropoutLimit[0]:
            print("Test dropout")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]-0.1, 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] -= 0.1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[0]+0.1) <= dropoutLimit[1]:
            print("Test dropout")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]+0.1, 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] += 0.1
                os.system("cp temp.h5 "+filenameModel)
                
                
        if (curentParam[1]-1) >= batchSizeLimit[0]:
            print("Test batchsize")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]-1))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[1]+1) <= batchSizeLimit[1]:
            print("Test batchsize")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]+1))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] += 1
                os.system("cp temp.h5 "+filenameModel)
                
        if (curentParam[2]-1) >= convolutionLayerLimit[0]:
            print("Test convolution")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]-1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[2]+1) <= convolutionLayerLimit[1]:
            print("Test convolution")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]+1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] += 1
                os.system("cp temp.h5 "+filenameModel)
                
        if (curentParam[3]-1) >= denseLayerLimit[0]:
            print("Test dense")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3]-1)),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[3]+1) <= denseLayerLimit[1]:
            print("Test dense")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3]+1)),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] += 1
                os.system("cp temp.h5 "+filenameModel)
                
    print('Dropout :', curentParam[0])
    print('Batch Size :', 2**curentParam[1])
    print('Convolutional layer :', 2**curentParam[2])
    print('Dense Layer :', 2**curentParam[3])
    os.system("rm temp.h5")
        
def createModel1(dropoutRate, convLayer, denseLayer):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu', input_shape=image.shape[1:]))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(denseLayer, activation='relu'))
    model.add(keras.layers.Dropout(dropoutRate))
    model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
    
    return model
    

In [5]:
optimizeHyperParameter(createModel1, '../models/KMNIST1.h5')

W0901 19:22:36.524161 140708058982208 deprecation.py:506] From /home/mathieu/miniconda3/envs/mlearning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0901 19:22:36.711377 140708058982208 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 1s 3ms/sample - loss: 8.1894 - acc: 0.0200 - val_loss: 7.5267 - val_acc: 0.0267
Epoch 2/50
350/350 [==============================] - 0s 1ms/sample - loss: 6.6661 - acc: 0.0400 - val_loss: 6.6041 - val_acc: 0.0133
Epoch 3/50
350/350 [==============================] - 0s 957us/sample - loss: 5.8770 - acc: 0.0229 - val_loss: 6.7577 - val_acc: 0.0667
Epoch 4/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.5134 - acc: 0.0257 - val_loss: 6.6430 - val_acc: 0.0667
Epoch 5/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.1381 - acc: 0.0371 - val_loss: 6.5271 - val_acc: 0.0667
Epoch 6/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.1135 - acc: 0.0429 - val_loss: 6.3724 - val_acc: 0.0667
Epoch 7/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.1169 - acc: 0.0200 - val_loss: 6.2945 - val_acc: 0.0667
Epoch 8/50
350/

W0901 19:22:43.444675 140708058982208 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 1s 3ms/sample - loss: 8.2185 - acc: 0.0314 - val_loss: 6.9563 - val_acc: 0.0667
Epoch 2/50
350/350 [==============================] - 0s 1ms/sample - loss: 6.5124 - acc: 0.0200 - val_loss: 6.7265 - val_acc: 0.0133
Epoch 3/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.5443 - acc: 0.0171 - val_loss: 6.0739 - val_acc: 0.0267
Epoch 4/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.0968 - acc: 0.0314 - val_loss: 5.9768 - val_acc: 0.0667
Epoch 5/50
350/350 [==============================] - 0s 884us/sample - loss: 4.8913 - acc: 0.0486 - val_loss: 6.0139 - val_acc: 0.0667
Epoch 6/50
350/350 [==============================] - 0s 1ms/sample - loss: 4.8214 - acc: 0.0457 - val_loss: 5.9119 - val_acc: 0.0667
Epoch 7/50
350/350 [==============================] - 0s 917us/sample - loss: 4.7670 - acc: 0.0486 - val_loss: 5.9126 - val_acc: 0.0667
Epoch 8/50
35

W0901 19:22:49.135025 140708058982208 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


...

Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 4s 11ms/sample - loss: 7.9859 - acc: 0.0229 - val_loss: 7.6085 - val_acc: 0.0267
Epoch 2/50
350/350 [==============================] - 1s 2ms/sample - loss: 6.1493 - acc: 0.0257 - val_loss: 6.7836 - val_acc: 0.0267
Epoch 3/50
350/350 [==============================] - 1s 2ms/sample - loss: 5.5031 - acc: 0.0229 - val_loss: 6.5152 - val_acc: 0.0267
Epoch 4/50
350/350 [==============================] - 1s 2ms/sample - loss: 5.2541 - acc: 0.0200 - val_loss: 6.0019 - val_acc: 0.0267
Epoch 5/50
350/350 [==============================] - 1s 2ms/sample - loss: 5.0824 - acc: 0.0343 - val_loss: 5.8641 - val_acc: 0.0667
Epoch 6/50
350/350 [==============================] - 1s 2ms/sample - loss: 4.8750 - acc: 0.0343 - val_loss: 5.8704 - val_acc: 0.0667
Epoch 7/50
350/350 [==============================] - 1s 4ms/sample - loss: 4.8581 - acc: 0.0257 - val_loss: 5.8879 - val_acc: 0.0667
Epoch 8/50
350/3

W0901 19:25:53.793033 140708058982208 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 4s 11ms/sample - loss: 8.1285 - acc: 0.0200 - val_loss: 7.0261 - val_acc: 0.0533
Epoch 2/50
350/350 [==============================] - 0s 1ms/sample - loss: 6.4994 - acc: 0.0571 - val_loss: 6.8555 - val_acc: 0.0133
Epoch 3/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.8068 - acc: 0.0314 - val_loss: 6.2351 - val_acc: 0.0133
Epoch 4/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.3351 - acc: 0.0257 - val_loss: 5.9098 - val_acc: 0.0667
Epoch 5/50
350/350 [==============================] - 0s 1ms/sample - loss: 5.2995 - acc: 0.0286 - val_loss: 5.7916 - val_acc: 0.0267
Epoch 6/50
350/350 [==============================] - 0s 1ms/sample - loss: 4.9010 - acc: 0.0486 - val_loss: 5.8485 - val_acc: 0.0667
Epoch 7/50
350/350 [==============================] - 1s 3ms/sample - loss: 4.8892 - acc: 0.0457 - val_loss: 5.8792 - val_acc: 0.0667
Epoch 8/50
350/3

W0901 19:26:04.242393 140708058982208 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 350 samples, validate on 75 samples
Epoch 1/50
350/350 [==============================] - 4s 13ms/sample - loss: 7.9101 - acc: 0.0229 - val_loss: 7.2425 - val_acc: 0.0133
Epoch 2/50
350/350 [==============================] - 1s 3ms/sample - loss: 6.2594 - acc: 0.0229 - val_loss: 6.8671 - val_acc: 0.0133
Epoch 3/50
350/350 [==============================] - 1s 2ms/sample - loss: 5.4435 - acc: 0.0286 - val_loss: 6.4236 - val_acc: 0.0667
Epoch 4/50
350/350 [==============================] - 1s 2ms/sample - loss: 5.2205 - acc: 0.0229 - val_loss: 6.0698 - val_acc: 0.0667
Epoch 5/50
350/350 [==============================] - 1s 2ms/sample - loss: 4.9888 - acc: 0.0429 - val_loss: 5.8662 - val_acc: 0.0667
Epoch 6/50
350/350 [==============================] - 1s 2ms/sample - loss: 4.9496 - acc: 0.0429 - val_loss: 5.8524 - val_acc: 0.0667
Epoch 7/50
350/350 [==============================] - 1s 2ms/sample - loss: 4.7630 - acc: 0.0343 - val_loss: 5.9995 - val_acc: 0.0667
Epoch 8/50
350/3

In [5]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
spatial_dropout2d_4 (Spatial (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
spatial_dropout2d_6 (Spatial (None, 12, 12, 64)       

In [6]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST5.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 575s 1ms/sample - loss: 3.7743 - acc: 0.3494 - val_loss: 2.4794 - val_acc: 0.5603
Epoch 2/5
478424/478424 [==============================] - 565s 1ms/sample - loss: 3.0402 - acc: 0.4542 - val_loss: 2.2473 - val_acc: 0.5994
Epoch 3/5
478424/478424 [==============================] - 563s 1ms/sample - loss: 2.9663 - acc: 0.4651 - val_loss: 2.2146 - val_acc: 0.6072
Epoch 4/5
478424/478424 [==============================] - 551s 1ms/sample - loss: 2.9680 - acc: 0.4662 - val_loss: 2.1369 - val_acc: 0.6186
Epoch 5/5
478424/478424 [==============================] - 587s 1ms/sample - loss: 3.0064 - acc: 0.4611 - val_loss: 2.2947 - val_acc: 0.5976


In [7]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_8 (Spatial (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_9 (Spatial (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_10 (Spatia (None, 12, 12, 32)       

In [8]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST2.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 337s 704us/sample - loss: 3.8546 - acc: 0.3332 - val_loss: 2.6197 - val_acc: 0.5320
Epoch 2/5
478424/478424 [==============================] - 344s 720us/sample - loss: 3.1551 - acc: 0.4318 - val_loss: 2.4001 - val_acc: 0.5679
Epoch 3/5
478424/478424 [==============================] - 357s 746us/sample - loss: 3.0688 - acc: 0.4440 - val_loss: 2.3285 - val_acc: 0.5762
Epoch 4/5
478424/478424 [==============================] - 395s 825us/sample - loss: 3.0786 - acc: 0.4433 - val_loss: 2.2172 - val_acc: 0.5955
Epoch 5/5
478424/478424 [==============================] - 342s 715us/sample - loss: 3.0537 - acc: 0.4469 - val_loss: 2.2711 - val_acc: 0.5890


In [9]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_12 (Spatia (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_13 (Spatia (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_14 (Spatia (None, 12, 12, 32)       

In [10]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST3.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 612s 1ms/sample - loss: 3.6970 - acc: 0.3668 - val_loss: 2.4662 - val_acc: 0.5723
Epoch 2/5
478424/478424 [==============================] - 630s 1ms/sample - loss: 2.9308 - acc: 0.4788 - val_loss: 2.1859 - val_acc: 0.6163
Epoch 3/5
478424/478424 [==============================] - 567s 1ms/sample - loss: 2.8272 - acc: 0.4934 - val_loss: 2.0808 - val_acc: 0.6329
Epoch 4/5
478424/478424 [==============================] - 556s 1ms/sample - loss: 2.7916 - acc: 0.4970 - val_loss: 2.0186 - val_acc: 0.6427
Epoch 5/5
478424/478424 [==============================] - 398s 832us/sample - loss: 2.7860 - acc: 0.4979 - val_loss: 2.0089 - val_acc: 0.6447


In [11]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_16 (Spatia (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_17 (Spatia (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_18 (Spatia (None, 12, 12, 32)       

In [12]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST4.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 333s 696us/sample - loss: 4.3671 - acc: 0.2399 - val_loss: 3.1050 - val_acc: 0.4387
Epoch 2/5
478424/478424 [==============================] - 330s 690us/sample - loss: 3.5764 - acc: 0.3544 - val_loss: 2.8102 - val_acc: 0.4892
Epoch 3/5
478424/478424 [==============================] - 333s 695us/sample - loss: 3.4758 - acc: 0.3690 - val_loss: 2.6842 - val_acc: 0.5058
Epoch 4/5
478424/478424 [==============================] - 337s 705us/sample - loss: 3.4492 - acc: 0.3726 - val_loss: 2.6470 - val_acc: 0.5119
Epoch 5/5
478424/478424 [==============================] - 330s 690us/sample - loss: 3.4554 - acc: 0.3718 - val_loss: 2.6807 - val_acc: 0.5082
